In [1]:
import os
import sys
import argparse
import torch
import cv2
import time
import re
import numpy as np
from ultralytics import YOLO
import math
from typing import List
import pandas as pd

/home/varun/anaconda3/envs/yolov8/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def compute_angle(A, B, C):
    # Vectors for BA and BC
    a = [A[0] - B[0], A[1] - B[1]]
    b = [C[0] - B[0], C[1] - B[1]]
    c = [A[0] - C[0], A[1] - C[1]]

    # Magnitudes (lengths) of BA, BC, and AC
    ma = math.sqrt(a[0] ** 2 + a[1] ** 2)
    mb = math.sqrt(b[0] ** 2 + b[1] ** 2)
    mc = math.sqrt(c[0] ** 2 + c[1] ** 2)

    print(ma)
    print(mb)
    print(mc)
    # Using the law of cosines to find the angle at B
    cosB = (ma ** 2 + mc ** 2 - mb ** 2) / (2 * ma * mc)

    # Angle in radians
    angle = math.acos(cosB)

    # Convert angle to degrees
    angle_deg = math.degrees(angle)

    return angle_deg

In [3]:
def calculate_pointer_angle(img_path):
    # Calculates the Pointer Angle
    # separate coordinates from box

    # Detect the gauge and get the pointer angle
    model1 = YOLO(model_path)
    results = model1(img_path)
    theta1 = None  # Initialize with a default value
    theta2 = None  # Initialize with a default value

    for result in results:
        keypoints_set = result.keypoints.xy.cpu().numpy()
        print(keypoints_set)

        # Check if keypoints_set is empty
        if keypoints_set.size == 0:
            print('No Key Point Detected')
            theta1 = 0
            theta2 = 45
            break

        # Check if there are at least 4 keypoints in the set
        if keypoints_set.shape[1] < 4:
            print('Not enough keypoints detected')
            break

        print(keypoints_set[0])
        A, B, C, D = keypoints_set[0][0], keypoints_set[0][1], keypoints_set[0][3], keypoints_set[0][2]
        print(keypoints_set[0])
        print(f"A is {A[1]}")
        print(f"B is {B[1]}")
        print(f"C is {C[1]}")

        theta1 = compute_angle(A, B, C)
        theta2 = 360 - compute_angle(B, D, C)

        print(f"Angle between keypoints A, B, and C is: {theta1:.2f} degrees")
        print(f"Angle between keypoints B, C, and D is: {theta2:.2f} degrees")
    return theta1, theta2

In [4]:
img_folder = "/home/varun/Desktop/KeyPointDetection/runs/pose/train16/data"

In [5]:
model_path = '/home/varun/Desktop/KeyPointDetection/runs/pose/train16/weights/best.pt'

In [6]:
import os
import pandas as pd
import matplotlib.pyplot as plt

In [7]:
angles_df = pd.DataFrame(columns=['File', 'Angle_ABC', 'Angle_BDC'])




In [8]:
data_list = []

In [9]:
for file_name in os.listdir(img_folder):
    if file_name.endswith(('.jpg', '.png', '.jpeg')):
        img_path = os.path.join(img_folder, file_name)
        image_output = calculate_pointer_angle(img_path)

        # Check if the angles are valid before appending to the list
        if all(angle is not None for angle in image_output):
            data_list.append({'File': file_name, 'Angle_ABC': image_output[0], 'Angle_BDC': image_output[1]})


image 1/1 /home/varun/Desktop/KeyPointDetection/runs/pose/train16/data/frame30.jpg: 384x640 1 KEYPOINT_SKELETON, 23.2ms
Speed: 4.4ms preprocess, 23.2ms inference, 2.1ms postprocess per image at shape (1, 3, 384, 640)
[[[     882.11      679.11]
  [     824.06      743.13]
  [     966.39      727.31]
  [     977.05       641.8]]]
[[     882.11      679.11]
 [     824.06      743.13]
 [     966.39      727.31]
 [     977.05       641.8]]
[[     882.11      679.11]
 [     824.06      743.13]
 [     966.39      727.31]
 [     977.05       641.8]]
A is 679.1102294921875
B is 743.1273193359375
C is 641.802734375
86.41834643129725
183.50382685686498
102.00916859345915
143.21395102691335
86.16571109885318
183.50382685686498
Angle between keypoints A, B, and C is: 153.65 degrees
Angle between keypoints B, C, and D is: 332.83 degrees

image 1/1 /home/varun/Desktop/KeyPointDetection/runs/pose/train16/data/frame144.jpg: 384x640 (no detections), 23.0ms
Speed: 1.7ms preprocess, 23.0ms inference, 0.

In [10]:
angles_df = pd.DataFrame(data_list)

In [11]:
import pandas as pd

angles_df['File'] = angles_df['File'].str.extract('(\d+)')
angles_df['File'] = pd.to_numeric(angles_df['File'])

     
     

In [12]:
angles_df = angles_df.sort_values(by='File').reset_index(drop=True)

In [13]:
print(angles_df)

    File   Angle_ABC   Angle_BDC
0      0  160.797324  300.287091
1      6  167.995676  305.420991
2     12  175.034189  310.538255
3     18  176.953824  316.537456
4     24  164.697412  324.241646
5     30  153.654315  332.826454
6     36    9.096182  247.421623
7     42    5.568567  217.716312
8     48  121.261756  351.445337
9     54  164.858300  322.699514
10    60  171.177912  305.465487
11    66  152.902304  296.664791
12    72  137.866688  291.499399
13    78  113.552338  268.122837
14    84  115.706027  275.980854
15    90  115.438160  275.767592
16    96   47.531419  195.684968
17   102   38.547923  189.038151
18   108   40.626762  188.253549
19   114   48.365445  191.103751
20   120   38.227994  189.273748
21   126    0.000000   45.000000
22   132    0.000000   45.000000
23   138    0.000000   45.000000
24   144    0.000000   45.000000
25   150    0.000000   45.000000
